In [4]:
pip install timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10.3

In [ ]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import ttach as tta

class RegionDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.df = df[df['filename'].apply(lambda x: os.path.exists(os.path.join(img_dir, x)))].reset_index(drop=True)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.iloc[idx]['filename'])
        image = np.array(Image.open(img_path).convert('RGB'))
        label = self.df.iloc[idx]['Region_ID'] - 1 if 'Region_ID' in self.df.columns else -1
        if self.transform:
            image = self.transform(image=image)['image']
        return image, label

train_df = pd.read_csv('/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/labels_train.csv')
val_df = pd.read_csv('/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/labels_val.csv')

test_filenames = sorted(os.listdir('/kaggle/input/images-test/images_test'))
test_df = pd.DataFrame({'filename': test_filenames})

train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2(),
])

train_dataset = RegionDataset(train_df, '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_train/images_train', transform=train_transform)
val_dataset = RegionDataset(val_df, '/kaggle/input/smai-25-sec-a-project-phase-2-region-id-prediction/images_val/images_val', transform=val_transform)
test_dataset = RegionDataset(test_df, '/kaggle/input/images-test/images_test', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=15)
model = model.to(device)

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, epsilon: float = 0.1, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction

    def forward(self, preds, target):
        n = preds.size()[-1]
        log_preds = F.log_softmax(preds, dim=-1)
        loss = -log_preds.sum(dim=-1)
        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        return self.epsilon * (loss / n) + (1 - self.epsilon) * nll

criterion = LabelSmoothingCrossEntropy(epsilon=0.1)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=25)

best_val_acc = 0
patience = 10
wait = 0

for epoch in range(25):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/25"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"\nValidation Accuracy: {val_acc:.4f} | Loss: {total_loss:.4f}\n")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("🛑 Early stopping triggered.")
            break

    scheduler.step()

model.load_state_dict(torch.load('best_model.pth'))
model.eval()
tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')

val_preds = []
with torch.no_grad():
    for images, _ in val_loader:
        images = images.to(device)
        outputs = tta_model(images)
        preds = outputs.argmax(1).cpu().numpy() + 1
        val_preds.extend(preds)

test_preds = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = tta_model(images)
        preds = outputs.argmax(1).cpu().numpy() + 1
        test_preds.extend(preds)

submission = pd.DataFrame({
    'id': list(range(738)),
    'Region_ID': val_preds + test_preds
})

# 13. Save
submission.to_csv('2022101108_1.csv', index=False)
print("✅ Final submission with test predictions saved as 2022101108_1.csv")


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Epoch 1/25: 100%|██████████| 205/205 [02:10<00:00,  1.57it/s]



Validation Accuracy: 0.7290 | Loss: 377.1703



Epoch 2/25: 100%|██████████| 205/205 [02:09<00:00,  1.59it/s]



Validation Accuracy: 0.8401 | Loss: 221.8977



...


Validation Accuracy: 0.9404 | Loss: 115.4069



Epoch 25/25: 100%|██████████| 205/205 [02:09<00:00,  1.59it/s]



Validation Accuracy: 0.9404 | Loss: 115.2195



/tmp/ipykernel_31/756783398.py:142: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


✅ Final submission with test predictions saved as 2022101108_1.csv
